<a href="https://colab.research.google.com/github/Piligrimich/ML/blob/main/GoogLeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import random_split

import torchvision.datasets as dset

from torchvision import transforms

import numpy as np
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
transform = transforms.Compose([
                                transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

train_data = dset.CIFAR100('./',download=True,train=True,transform=transform)
test_data = dset.CIFAR100('./',download=True,train=False,transform=transform)

  0%|          | 0/169001437 [00:00<?, ?it/s]

Extracting ./cifar-100-python.tar.gz to ./
Files already downloaded and verified


In [3]:
batch_size = 64

data_size = train_data.data.shape[0]
validation_split = .2
split = int(np.floor(data_size*validation_split))

indices = list(range(data_size))

np.random.shuffle(indices)

train_indices, val_indices = indices[split:],indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,sampler=train_sampler)
val_loader = torch.utils.data.DataLoader(train_data,batch_size=batch_size,sampler=val_sampler)


In [4]:
from torchvision import models

In [5]:
model = models.googlenet(pretrained=True)

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


  0%|          | 0.00/49.7M [00:00<?, ?B/s]

In [7]:
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs,100)

In [82]:
num_ftrs

1024

In [81]:
for name, child in model.named_children():
  print(name)

conv1
maxpool1
conv2
conv3
maxpool2
inception3a
inception3b
maxpool3
inception4a
inception4b
inception4c
inception4d
inception4e
maxpool4
inception5a
inception5b
avgpool
dropout
fc


In [8]:
if torch.cuda.is_available:
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

print(device)

cuda


In [80]:
def fit(model,train_loader,val_loader,loss,optimizer,tol_epochs,accuracy_tol):
    epoch = 0
    
    #loss_sum = 0
    keep_training = True
    last_accuracies =[]

    while keep_training == True:
      model.train()
      correct_samples = 0
      total_samples = 0

      for x,y in train_loader:
        x,y = x.to(device),y.to(device)
        
        outputs = model(x)
        loss_value = loss(outputs,y)

        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

        _,indices = torch.max(outputs,dim=1)
        correct_samples += torch.sum(indices == y)
        total_samples += y.shape[0]
        #loss_sum += loss_value
      val_accuracy = validationAccuracy(model,val_loader)
      
      print("Epoch: %f  Accuracy %f  Val accuracy %f "%(epoch+1,(correct_samples/total_samples),val_accuracy))
      epoch+=1
      last_accuracies.append(val_accuracy)

      if len(last_accuracies) > tol_epochs:
        last_accuracies.pop(0)
      
      if len(last_accuracies) == tol_epochs:
        accuracy_difference = max(last_accuracies)-min(last_accuracies)
        if accuracy_difference <= accuracy_tol:
          keep_training = False


def validationAccuracy(model,val_loader):
  model.eval()
  correct_samples = 0
  total_samples = 0
  for i_step,(x,y) in enumerate(val_loader):
    x,y = x.to(device),y.to(device)
    predicts = model(x)

    
    _, indices = torch.max(predicts,dim=1)
    correct_samples += torch.sum(indices == y)
    total_samples += y.shape[0]

  return correct_samples/total_samples

In [10]:
optimizer = torch.optim.Adam(model.parameters())
loss = nn.CrossEntropyLoss()

In [11]:
model.to(device)

GoogLeNet(
  (conv1): BasicConv2d(
    (conv): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool1): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (conv2): BasicConv2d(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): BasicConv2d(
    (conv): Conv2d(64, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(192, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
  )
  (maxpool2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=True)
  (inception3a): Inception(
    (branch1): BasicConv2d(
      (conv): Conv2d(192, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=0.001, momentum=0.1, affine=True, track

In [12]:
fit(model,train_loader,val_loader,loss,optimizer,accuracy_tol=0.02,tol_epochs=10)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


Epoch: 1.000000  Accuracy 0.254200  Val accuracy 0.340800 
Epoch: 2.000000  Accuracy 0.405400  Val accuracy 0.415200 
Epoch: 3.000000  Accuracy 0.483275  Val accuracy 0.453200 
Epoch: 4.000000  Accuracy 0.534000  Val accuracy 0.457400 
Epoch: 5.000000  Accuracy 0.579000  Val accuracy 0.480800 
Epoch: 6.000000  Accuracy 0.627850  Val accuracy 0.490000 
Epoch: 7.000000  Accuracy 0.663100  Val accuracy 0.495900 
Epoch: 8.000000  Accuracy 0.697775  Val accuracy 0.484600 
Epoch: 9.000000  Accuracy 0.729175  Val accuracy 0.482400 
Epoch: 10.000000  Accuracy 0.754375  Val accuracy 0.496400 
Epoch: 11.000000  Accuracy 0.779425  Val accuracy 0.493400 
Epoch: 12.000000  Accuracy 0.802275  Val accuracy 0.496400 
Epoch: 13.000000  Accuracy 0.821725  Val accuracy 0.499800 
Epoch: 14.000000  Accuracy 0.831175  Val accuracy 0.501600 
Epoch: 15.000000  Accuracy 0.846250  Val accuracy 0.500100 


In [65]:
import sklearn.metrics

In [76]:
batch_size = 64

data_size = test_data.data.shape[0]

indices = list(range(data_size))

np.random.shuffle(indices)

test_indices = indices[:]

test_sampler = SubsetRandomSampler(test_indices)

test_loader = torch.utils.data.DataLoader(test_data,batch_size=batch_size,sampler=test_sampler)


In [78]:
def metrics(model,data):
  model.eval()
  actual_data = []
  predicted_data = []
  for X,Y in data:
    X = X.to(device)
    pred = model(X)
    _,indices = torch.max(pred,dim=1)
    indices = indices.to('cpu')
    indices = indices.detach().numpy()
    Y = Y.detach().numpy()
    for i in range(len(X)):
      actual_data.append(Y[i])
      predicted_data.append(indices[i])

  all_classes = train_data.classes

  print(sklearn.metrics.classification_report(actual_data,predicted_data,target_names=all_classes,))


In [79]:
metrics(model,test_loader)

/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"


               precision    recall  f1-score   support

        apple       0.90      0.64      0.75       100
aquarium_fish       0.67      0.62      0.65       100
         baby       0.22      0.37      0.28       100
         bear       0.27      0.35      0.30       100
       beaver       0.29      0.30      0.29       100
          bed       0.54      0.60      0.57       100
          bee       0.53      0.62      0.57       100
       beetle       0.63      0.53      0.58       100
      bicycle       0.70      0.76      0.73       100
       bottle       0.55      0.64      0.59       100
         bowl       0.46      0.38      0.42       100
          boy       0.27      0.40      0.32       100
       bridge       0.44      0.58      0.50       100
          bus       0.57      0.38      0.46       100
    butterfly       0.45      0.40      0.42       100
        camel       0.51      0.35      0.42       100
          can       0.56      0.52      0.54       100
       ca